In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cvxpy as cp
import seaborn as sns

# Defining the Game

In [2]:
states = [(h, a, s) for h in [0, 1, 2, 3, 4] for a in [0, 1, 2, 3] for s in [0, 1, 2]]
actions = [(state, 'NOOP') for state in states if state[0] == 0] + \
            [(state, 'SHOOT') for state in states if state[0] != 0 and state[1] != 0 and state[2] != 0] + \
            [(state, 'DODGE') for state in states if state[0] != 0 and state[2] != 0] + \
            [(state, 'RECHARGE') for state in states if state[0] != 0 and state[2] != 2]
print('States count:', len(states), '& Actions count:', len(actions))
print('*   States are:', states)
print('*   Actions are:', actions)

States count: 60 & Actions count: 100
*   States are: [(0, 0, 0), (0, 0, 1), (0, 0, 2), (0, 1, 0), (0, 1, 1), (0, 1, 2), (0, 2, 0), (0, 2, 1), (0, 2, 2), (0, 3, 0), (0, 3, 1), (0, 3, 2), (1, 0, 0), (1, 0, 1), (1, 0, 2), (1, 1, 0), (1, 1, 1), (1, 1, 2), (1, 2, 0), (1, 2, 1), (1, 2, 2), (1, 3, 0), (1, 3, 1), (1, 3, 2), (2, 0, 0), (2, 0, 1), (2, 0, 2), (2, 1, 0), (2, 1, 1), (2, 1, 2), (2, 2, 0), (2, 2, 1), (2, 2, 2), (2, 3, 0), (2, 3, 1), (2, 3, 2), (3, 0, 0), (3, 0, 1), (3, 0, 2), (3, 1, 0), (3, 1, 1), (3, 1, 2), (3, 2, 0), (3, 2, 1), (3, 2, 2), (3, 3, 0), (3, 3, 1), (3, 3, 2), (4, 0, 0), (4, 0, 1), (4, 0, 2), (4, 1, 0), (4, 1, 1), (4, 1, 2), (4, 2, 0), (4, 2, 1), (4, 2, 2), (4, 3, 0), (4, 3, 1), (4, 3, 2)]
*   Actions are: [((0, 0, 0), 'NOOP'), ((0, 0, 1), 'NOOP'), ((0, 0, 2), 'NOOP'), ((0, 1, 0), 'NOOP'), ((0, 1, 1), 'NOOP'), ((0, 1, 2), 'NOOP'), ((0, 2, 0), 'NOOP'), ((0, 2, 1), 'NOOP'), ((0, 2, 2), 'NOOP'), ((0, 3, 0), 'NOOP'), ((0, 3, 1), 'NOOP'), ((0, 3, 2), 'NOOP'), ((1, 1, 1), 'SH

In [3]:
def probability(state, action):
    (hf, af, sf) = state
    (hi, ai, si), act = action
    if act == 'SHOOT':
        if si - 1 == sf and (hi - 1 == hf or hi == hf) and ai - 1 == af:
            return 0.5
        else:
            return 0
    elif act == 'DODGE':
        if hi != hf:
            return 0
        if ai == 3 and af == 3:
            arrow_mult = 1.0
        elif ai + 1 == af:
            arrow_mult = 0.8
        elif ai == af:
            arrow_mult = 0.2
        else:
            arrow_mult = 0.0
        if si == 2 and sf == 1:
            stamn_mult = 0.8
        elif si == 2 and sf == 0:
            stamn_mult = 0.2
        elif si == 1 and sf == 0:
            stamn_mult = 1.0
        else:
            stamn_mult = 0.0
        return arrow_mult * stamn_mult
    elif act == 'RECHARGE':
        if hi != hf or ai != af:
            return 0.0
        elif si + 1 == sf:
            return 0.8
        elif si == sf:
            return 0.2 # Removing probabilities of the self loop
        else:
            return 0.0
    elif act == 'NOOP':
        if (hi, ai, si) == (hf, af, sf):
            return 0.0
        else:
            return 0.0
    else:
        raise ValueError("Invalid Action specified: " + act)

def reward(action):
    (hi, ai, si), act = action
    if act == 'NOOP':
        return 0.0
    else:
        return -10.0 # TODO: Determine using team number

def initial(state):
    return 1.0 if state == (4, 3, 2) else 0.0

def delta(state, action):
    (hf, af, sf) = state
    (hi, ai, si), act = action
    if hf == hi and af == ai and sf == si:
        return 1.0
    else:
        return 0.0

print('On action', actions[0], 'going to', states[0])
print('    probability:', probability(states[0], actions[0]))
print('    reward:', reward(actions[0]))
print('    alpha:', initial(states[0]))
print('    kronecker delta:', delta(states[0], actions[0]))

On action ((0, 0, 0), 'NOOP') going to (0, 0, 0)
    probability: 0.0
    reward: 0.0
    alpha: 0.0
    kronecker delta: 1.0


# Linear Programming Solver

In [4]:
def solve(a, b, r):
    """
    Solves the Convex optimization problem 
    maximize rx under the constraint ax = b 
    """
    states, doables = a.shape[0], a.shape[1]
    x = cp.Variable(doables)
    
    objective = cp.Maximize(r @ x)
    constraints = [a @ x == b, x >= 0]
    prob = cp.Problem(objective, constraints)

    result = prob.solve()
    print('Solved with status', prob.status, 'and value', prob.value)
    return x.value, prob.value


# Tiny little test to check if everything is working fine
_a = np.array([[1, 1], [0, 1]])
_b = np.array([20, 10])
_r = np.array([1, 1])
# max(x + y) under x + y = 20 and y = 10
print("Variables and Values:", solve(_a, _b, _r))

Solved with status optimal and value 20.0
Variables and Values: (array([10., 10.]), 20.0)


In [5]:
n_states, n_actions = len(states), len(actions)
a = np.zeros(shape=(n_states, n_actions))
r = np.zeros(shape=(n_actions))
b = np.zeros(shape=(n_states))

for i in range(n_states):
    for j in range(n_actions):
        a[i, j] = delta(states[i], actions[j]) - probability(states[i], actions[j])
for i in range(n_states):
    b[i] = initial(states[i])
for i in range(n_actions):
    r[i] = reward(actions[i])
    
print('A:', a)
print('Alpha:', b)
print('r:', r)

A: [[ 1.   0.   0.  ...  0.   0.   0. ]
 [ 0.   1.   0.  ...  0.   0.   0. ]
 [ 0.   0.   1.  ...  0.   0.   0. ]
 ...
 [ 0.   0.   0.  ...  0.   0.8  0. ]
 [ 0.   0.   0.  ...  0.  -0.8  0.8]
 [ 0.   0.   0.  ...  0.   0.  -0.8]]
Alpha: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
r: [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. -10. -10.
 -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.
 -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.
 -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.
 -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.
 -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.
 -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10. -10.
 -10. -10.]


The following function just dumps the A matrix for checking manually against expected probabilities

In [6]:
def dump_matrix():
    import json
    _a_test = a.tolist()
    with open('matrix_sol.test', 'w') as f:
        f.write(json.dumps(_a_test))
        
dump_matrix()

# Solving and Answering

In [7]:
x, v = solve(a, b, r)
print('Solution:', x)

Solved with status optimal and value -295.62499999556303
Solution: [ 1.00000000e+00  4.67596490e-11  2.62815741e-13  4.26293184e-11
  2.23713367e-11  2.62815741e-13 -3.38149282e-12  1.74638162e-11
  2.62815741e-13  2.62815741e-13  2.62815741e-13  2.62815741e-13
  2.00000000e+00  9.31952762e-11  8.50658718e-11  4.44081997e-11
 -7.09897351e-12  3.45439997e-11  4.42418857e-01  5.26411716e-01
  4.00009629e-01  6.31159798e-01  1.69850016e-10  1.94067330e-10
  3.14166700e-01  3.61092059e-01  2.69002682e-01  4.25701218e-01
  2.72087877e-01  3.57949465e-01  1.24201960e-01  1.38980213e-01
  1.86732562e-01  2.55345224e-01  1.51773354e-01  1.14296669e+00
  1.85551911e+00  1.24890020e-10 -1.76568476e-11 -3.06727073e-11
  6.07880513e-11 -2.65434066e-11 -5.21076341e-11 -5.34016789e-11
  1.02755583e+00  2.78622646e-10  8.95188445e-01  5.68635535e-11
  4.55692025e-10 -1.24039703e-13 -4.69502835e-11 -4.19255663e-11
  5.86525583e-01  2.73211623e-10  4.52940687e-01  2.18617487e-11
  7.87546677e-01  4.466

In [8]:
policy = {state: ('VOID', -1e100) for state in states}
for i in range(len(actions)):
    _s, _a = actions[i]
    if policy[_s][1] < x[i]:
        policy[_s] = (_a, x[i])
print(policy)

{(0, 0, 0): ('NOOP', 0.9999999998699242), (0, 0, 1): ('NOOP', 4.6759648999919543e-11), (0, 0, 2): ('NOOP', 2.6281574140062107e-13), (0, 1, 0): ('NOOP', 4.2629318351434804e-11), (0, 1, 1): ('NOOP', 2.2371336698614843e-11), (0, 1, 2): ('NOOP', 2.6281574140062107e-13), (0, 2, 0): ('NOOP', -3.3814928249998106e-12), (0, 2, 1): ('NOOP', 1.7463816204340943e-11), (0, 2, 2): ('NOOP', 2.6281574140062107e-13), (0, 3, 0): ('NOOP', 2.6281574140062107e-13), (0, 3, 1): ('NOOP', 2.6281574140062107e-13), (0, 3, 2): ('NOOP', 2.6281574140062107e-13), (1, 0, 0): ('RECHARGE', 1.9903915624547792), (1, 0, 1): ('DODGE', 1.8555191078568578), (1, 0, 2): ('DODGE', 1.248900201076138e-10), (1, 1, 0): ('RECHARGE', 2.105525126126352), (1, 1, 1): ('SHOOT', 1.9999999997398465), (1, 1, 2): ('SHOOT', 9.319527621799763e-11), (1, 2, 0): ('RECHARGE', 9.145945352466877e-11), (1, 2, 1): ('SHOOT', 8.506587181271706e-11), (1, 2, 2): ('SHOOT', 4.440819972348678e-11), (1, 3, 0): ('RECHARGE', -2.32990417539963e-11), (1, 3, 1): ('

In [9]:
result = {
    'a': a.tolist(),
    'r': r.tolist(),
    'alpha': b.tolist(),
    'x': x.tolist(),
    'policy': [[list(key), value[0]] for key, value in policy.items()],
    'objective': v
}

In [10]:
import json
with open('answers.json', 'w') as f:
    f.write(json.dumps(result))